### 🧠 Cell 1 — Setup (imports + reproducibility)
Use TensorFlow/Keras and a few helpers.
numpy (np) → Handles numerical data and arrays.

tensorflow (tf) → Framework for building and training neural networks.

Sequential → Lets you build models layer-by-layer.

Embedding → Converts words into dense numeric vectors (word meanings).

SimpleRNN → Learns sequence patterns; remembers previous words.

Dense → Output layer; predicts the next word from all vocabulary words.

Tokenizer → Turns text into sequences of integers.

pad_sequences → Pads all sequences to the same length with zeros.

os, random → OS interaction and random number generation.
 The seed is optional—just helps make results repeatable.

In [1]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os, random

# (optional) reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)


### 🧾 Cell 2 — Training text

* The variable **`text`** stores a **large paragraph** that will be used as **training data**.
* This text gives the RNN examples of how words follow each other.
* The model will learn from these patterns to **predict the next word** in a sentence.
* You can replace this paragraph with **any long text** (e.g., a story or article) to improve accuracy.


In [2]:

text = """
Deep learning is a branch of machine learning that uses neural networks with many layers.
These networks can automatically learn features from large amounts of data, allowing them
to perform tasks like image recognition, speech understanding, and natural language processing.
Recurrent neural networks, or RNNs, are designed to handle sequential data by maintaining
a hidden state that captures information about previous inputs. This makes them powerful
for text prediction, translation, and time series forecasting. When trained on large corpora,
they can generate realistic text and predict the next word or phrase in a sequence.
"""


### 🔢 Cell 3 — Tokenize the text
Tokenizer() → creates a tool to convert words into numeric IDs.

fit_on_texts([text]) → reads the text and builds a vocabulary (word-to-number mapping).

total_words → counts how many unique words are in the text.

+1 is added for padding (used later to make all sequences equal in length).

In [3]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1  # +1 for padding index


### 📊 Cell 4 — Build n-gram training sequences
input_sequences = [] → creates an empty list to store training samples.

text.split('.') → splits the paragraph into sentences.

tokenizer.texts_to_sequences() → converts each sentence into a list of word IDs.

Inner loop → creates n-gram sequences, e.g.

[deep, learning, is] → [1, 2, 3], [1, 2, 3, 4], etc.

These sequences teach the model the order of words.

len(input_sequences) → shows how many training examples were created.

In [4]:

input_sequences = []

# Split by sentence to keep context tidy (simple heuristic)
for line in text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    # Create incremental n-grams
    for i in range(1, len(token_list)):
        n_gram_seq = token_list[:i+1]
        input_sequences.append(n_gram_seq)

len(input_sequences)


89

### 🧮 Cell 5 — Pad sequences and split X/y
max_seq_len → finds the length of the longest sequence.

pad_sequences(..., padding='pre') → adds zeros at the start so all sequences are equal in length.

X → all words except the last one (input to the model).

y → the last word in each sequence (the target/output).

to_categorical → converts y into one-hot encoded vectors for classification.

Output shapes show the size of input data, target data, vocabulary, and sequence length.

In [5]:

max_seq_len = max(len(seq) for seq in input_sequences)

# Left-pad with zeros
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')

# X: all but last token, y: last token
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# One-hot encode labels
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

X.shape, y.shape, max_seq_len, total_words


((89, 24), (89, 78), 25, 78)

### 🧩 Cell 6 — Define the RNN model
Sequential([...]) → builds the model layer-by-layer.

Embedding → turns each word ID into a 50-dimensional vector (captures meaning).

SimpleRNN(150) → learns word order and context using 150 memory units.

Dense(total_words, activation='softmax') → predicts the probability of each word in the vocabulary as the next word.

compile(...) → sets up the model for training:

Loss: categorical_crossentropy for multi-class prediction.

Optimizer: adam adjusts learning automatically.

Metric: accuracy to track performance.

In [6]:

model = Sequential([
    Embedding(input_dim=total_words, output_dim=50, input_length=max_seq_len - 1),
    SimpleRNN(150),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


c:\Anaconda3\envs\MachineLearning\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


### ⚙️ Cell 7 — Train
model.fit(X, y, epochs=200, verbose=1) → trains the model using input X and target y.

epochs=200 → runs through the dataset 200 times to improve learning.

verbose=1 → shows training progress after each epoch (loss and accuracy).

In [11]:

history = model.fit(X, y, epochs=200, verbose=1)


Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 0.0027
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 0.0027 
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 1.0000 - loss: 0.0026
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 0.0026 
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 0.0026
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 0.0026
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 0.0026
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 0.0026
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 0.0026
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 0.0026
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 0.0025
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - 

### 🔮 Cell 8 — Helper: next-word prediction
Purpose: Predicts the next word for a given input phrase.

texts_to_sequences → converts the seed text into numeric tokens.

pad_sequences → pads to match the model’s input length.

model.predict() → outputs probabilities for all words.

np.argmax(...) → finds the index of the most likely next word.

Loop → converts that index back to the actual word using the tokenizer’s dictionary.

In [8]:

def predict_next_word(model, tokenizer, text_seed, max_seq_len):
    token_list = tokenizer.texts_to_sequences([text_seed])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_len - 1, padding='pre')
    predicted_id = int(np.argmax(model.predict(token_list, verbose=0), axis=-1)[0])

    # id → word
    for word, idx in tokenizer.word_index.items():
        if idx == predicted_id:
            return word
    return None  # fallback


### 🧾 Cell 9 — Try it
seed_text → starting phrase given to the model.

predict_next_word(...) → predicts the next most likely word.

Print statement → shows the input and predicted next word.

complete_text() → keeps predicting and adding words to form a short generated sentence.

Output → displays how the model continues a given phrase.

In [9]:

seed_text = "deep learning is"
next_word = predict_next_word(model, tokenizer, seed_text, max_seq_len)
print(f"Seed: {seed_text}\nNext word → {next_word}")

# (optional) generate multiple words
def complete_text(seed, n_words=5):
    s = seed
    for _ in range(n_words):
        w = predict_next_word(model, tokenizer, s, max_seq_len)
        if not w: break
        s += " " + w
    return s

print("\nGenerated:", complete_text("recurrent neural networks", n_words=5))


Seed: deep learning is
Next word → a

Generated: recurrent neural networks or rnns are designed to
